In [1]:
import json
import nltk
#nltk.download('punkt_tab')
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
import random
import pandas as pd
import warnings
#!pip install tensorflow
import tensorflow.keras.models
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Activation, Input, Flatten
from tensorflow.keras.models import Sequential
from textattack.augmentation import EasyDataAugmenter

In [3]:
try:
    path = 'C:/Users/61450/Desktop/New folder/Chatbot/intents.json'
except FileNotFoundError as e: 
    print(f"Error: {e}") 
    
with open(path, 'r') as file:
   uni_dt = json.load(file)

In [5]:
[uni_dt['intents'][i]['intent'] for i in range(len(uni_dt['intents']))]

['greeting',
 'goodbye',
 'creator',
 'name',
 'hours',
 'number',
 'course',
 'fees',
 'location',
 'hostel',
 'event',
 'document',
 'floors',
 'syllabus',
 'library',
 'infrastructure',
 'canteen',
 'menu',
 'placement',
 'ithod',
 'computerhod',
 'extchod',
 'principal',
 'sem',
 'admission',
 'scholarship',
 'facilities',
 'college intake',
 'uniform',
 'committee',
 'random',
 'swear',
 'vacation',
 'sports',
 'salutaion',
 'task',
 'ragging',
 'hod',
 'Mess Timetable']

In [7]:
# The loaded data has information about university queries. The file contains a list of intents with tags, text(user input), 
# responses, extension, context, entity type and entities. The data is organized as a nested dictionary where 'intents' is the main key.
# Each dictionary value is also a dictionary where the main keys are 'intent', 'text' and 'responses'.

uni_dt['intents'][0:2]

[{'intent': 'greeting',
  'text': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'extension': {'function': '', 'entities': False, 'responses': []},
  'context': {'in': '', 'out': 'GreetingUserRequest', 'clear': False},
  'entityType': 'NA',
  'entities': []},
 {'intent': 'goodbye',
  'text': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'extension': {'function': '', 'entities': False, 'responses': []},
  'context': {'in': '', 'out': 'LeavingUserRequest', 'clear': False},
  'entityType': 'NA',
  'entities': []}]

In [9]:
#!pip install textattack

In [65]:
#The tags and user input text are reorganized as dataframe for clear representation of tag/intent for each user input.

eda_aug = EasyDataAugmenter()

user_in = []
tag_in = []
for i in range(len(uni_dt['intents'])):
    for k in uni_dt['intents'][i]['text']:
        user_in.append([k])
        user_in.append(eda_aug.augment(k))
        l = len(eda_aug.augment(k))
        tag_in.append(uni_dt['intents'][i]['intent'])
        for x in range(l):
            tag_in.append(uni_dt['intents'][i]['intent'])

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\61450\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [66]:
user_input = [x for m in user_in for x in m]
len(user_input)

1955

In [69]:
len(tag_in)

1955

In [71]:
uni_data_in = pd.DataFrame({'tags':tag_in, 'user_input':user_input})
uni_data_in

,tags,user_input
0,greeting,Hi
1,greeting,Hi
2,greeting,hello Hi
3,greeting,how-do-you-do
4,greeting,How are you?
...,...,...
1950,Mess Timetable,Academic Calander
1951,Mess Timetable,Calander Academic
1952,Mess Timetable,donnish Calander
1953,Mess Timetable,Academic


In [73]:
#The responses and corresponding tags are also reorganized as dataframe for clear representation of tag/intent for each chatbot response.

tag_out = []
resp = []
for i in range(len(uni_dt['intents'])):
    for k in uni_dt['intents'][i]['responses']:
        resp.append(k)
        tag_out.append(uni_dt['intents'][i]['intent'])

uni_data_out = pd.DataFrame({'tags':tag_out, 'responses':resp})
uni_data_out

,tags,responses
0,greeting,Hello!
1,greeting,Good to see you again!
2,greeting,"Hi there, how can I help?"
3,goodbye,Sad to see you go :(
4,goodbye,Talk to you later
5,goodbye,Goodbye!
6,goodbye,Come back soon
7,creator,College students
8,name,You can call me Mind Reader.
9,name,I'm Mind Reader


In [75]:
# A function is created for preprocessing user text. Stop words have not been excluded as the user content has very short messages with commonly used
# words. Removal of any stopword may remove important text from input patterns. Each user text is converted to lower case. Punctuation marks are removed 
# and the text is subjected to lemmatization for extraction of correct root words.

def preprocess(line, remove_stopwords):
    line = line.lower()
    line = re.sub(r'[^\w\s]', ' ',line)
    line = line.split()
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        line = [word for word in line if word not in stop_words]
    word_lem = WordNetLemmatizer()
    line = [word_lem.lemmatize(i) for i in line]
    line = ' '.join(line)
    return line

In [77]:
#The tags are the labelled responses and are encoded into integer labels for training the neural network model.

label_en = LabelEncoder()
labels = label_en.fit_transform(uni_data_in['tags'])
labels

array([15, 15, 15, ...,  0,  0,  0])

In [121]:
# The user input corpus is subjected to preprocessing. The preprocessed text is tokenized to extract a list of unique words
# and the size of vocabulary.

word_set = []
user_sentences = []
for sentence in uni_data_in['user_input']:
    sent = preprocess(sentence, remove_stopwords=True)
    word = word_tokenize(sent)
    user_sentences.append(sent)
    word_set.append(word)

vocab = [m for s in word_set for m in s]
vocab = set(vocab)
vocab_size = len(vocab)

In [122]:
# Input data is clean user input and output is the integer label of the corresponding tag.

proc_data_in = pd.DataFrame({'tags':labels, 'user_input':user_sentences})
proc_data_in

,tags,user_input
0,15,hi
1,15,hi
2,15,hello hi
3,15,
4,15,
...,...,...
1950,0,academic calander
1951,0,calander academic
1952,0,donnish calander
1953,0,academic


In [123]:
# The tokenizer function is used to convert the input corpus into sequence vectors.

tokens = Tokenizer(num_words=vocab_size)
tokens.fit_on_texts(proc_data_in['user_input'])
encoded_text = tokens.texts_to_sequences(proc_data_in['user_input'])

In [124]:
#The maximumm length of a user input message is extracted to create sequence vectors of uniform lengths

max_length = 0
for sentence in user_sentences:
    if(max_length < len(sentence.split())):
        max_length = len(sentence.split())

In [129]:
# The sequence vectors are padded to the maximum length.

padded_data = pad_sequences(encoded_text, maxlen=max_length, padding='post', value=0)
padded_data

array([[164,   0,   0, ...,   0,   0,   0],
       [164,   0,   0, ...,   0,   0,   0],
       [165, 164,   0, ...,   0,   0,   0],
       ...,
       [565, 221,   0, ...,   0,   0,   0],
       [220,   0,   0, ...,   0,   0,   0],
       [220, 221,   0, ...,   0,   0,   0]])

In [131]:
# Reference used for model building: https://www.kaggle.com/code/uzzivirus/simple-chatbot-using-neural-networks.
# The neural network model has an input of length equal to maximum length. 
# First layer is the embedding layer followed by 3 dense layers.
# The ouput layer has length equal to the number of unique tags. The activation function used is 'Relu' and 'softmax' is used
# for multi class classfication.
warnings.filterwarnings('ignore')

model = Sequential()
model.add(Input(shape=(max_length,)))
model.add(Embedding(input_dim=vocab_size,output_dim=8, input_length=(max_length,)))
model.add(Flatten())
model.add(Dense(units= 64, activation='relu'))
model.add(Dense(units= 64, activation='relu'))
model.add(Dense(units= len(set(labels)), activation='softmax'))

In [133]:
# The optizer used is adam optimiser. Sparse categorical cross entropy is used due to multi class 
# classification and also because the targets are labelled as integers. 

model.compile(loss = "sparse_categorical_crossentropy", optimizer ='adam', metrics=['accuracy'])

In [135]:
model.fit(padded_data, labels, epochs=150, verbose=1, validation_split=0.1)

Epoch 1/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.0522 - loss: 3.6395 - val_accuracy: 0.0000e+00 - val_loss: 4.4999
Epoch 2/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0875 - loss: 3.3761 - val_accuracy: 0.0000e+00 - val_loss: 6.5425
Epoch 3/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2362 - loss: 3.0279 - val_accuracy: 0.0000e+00 - val_loss: 7.3006
Epoch 4/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3371 - loss: 2.4216 - val_accuracy: 0.0000e+00 - val_loss: 9.0838
Epoch 5/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4833 - loss: 1.9421 - val_accuracy: 0.0000e+00 - val_loss: 11.3516
Epoch 6/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6031 - loss: 1.5058 - val_accuracy: 0.0000e+00 - val_loss: 13.1706
Epoch 7/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7188 - loss: 1.1769 - val_accuracy: 0.0000e+00 - val_loss: 15.0226
Epoch 8/150
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7921 - loss: 0

In [137]:
# The argument passed to the chatbot function is the user query. The result returned by the function is the chatbot response.

def chatbot(input_sentence):
    if not input_sentence.strip():
        return 'Sorry. I did not understand. Could you please rephrase?'
        
    processed_text = preprocess(input_sentence, remove_stopwords=True)  #function preprocess defined previously
    
    txt = tokens.texts_to_sequences(processed_text.split())
    coded_txt = [x for y in txt for x in y]
    padded_data = pad_sequences([coded_txt], maxlen=max_length, padding='post', value=0)  # processed text converted into padded sequence
    
    out = (model.predict(padded_data)).argmax()

    #adding a confidence threshold of 40%
    
    if out>0.4:
        label_out = label_en.inverse_transform([out]) #convert the predicted label into the corresponding tag
        label_out = ''.join(label_out)
        response = uni_data_out[uni_data_out['tags']==label_out]['responses']  #select list of responses corresponding to the tag
        response = random.choice(list(response)) #random selection of one of the respones for the given tag
        return response
        
    else:
        return 'Sorry. Could not understand. Please rephrase.'

    if response.empty: 
        return "Sorry, I couldn't find a suitable response."

In [139]:
input_no = int(input())  #maximum integer input 411

#user input
print('User: ', uni_data_in['user_input'][input_no])

# chatbot response
print('Chatbot: ', chatbot(uni_data_in['user_input'][input_no]))

 567


User:  fees first for year
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Chatbot:  For Fee detail visit <a target="_blank" href="LINK"> here</a>


In [145]:
chatbot('what are the hostel facilities')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


'For hostel detail visit <a target="_blank" href="ADD YOUR HOSTEL DETAIL PDF LINK OR ANY INFORMATION LINK OR ADD YOU OWN ANSWERS"> here</a>'